# Triplet excitations
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/response/triplet_excitations/)

In this tutorial, we will calculate triplet excitations for methane with time-propagation and Casida methods.


In [ ]:
import subprocess

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!mkdir 5_Triplet_excitations

In [ ]:
cd 5_Triplet_excitations/

## Time-propagation

### Ground-state
We begin with a spin-polarized calculation of the ground-state, as before but with <code><a href=https://www.octopus-code.org/documentation//13/variables/states/spincomponents>SpinComponents</a> = spin_polarized</code> specified now.


In [ ]:
%%writefile inp

stdout = 'stdout_gs_triplet.txt'
stderr = 'stderr_gs_triplet.txt'
CalculationMode = gs
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

SpinComponents = spin_polarized

In [ ]:
!octopus

You can verify that the results are identical in detail to the non-spin-polarized calculation since this is a non-magnetic system.

### Time-propagation

Next, we perform the time-propagation using the following input file:


In [ ]:
%%writefile inp

stdout = 'stdout_td_triplet.txt'
stderr = 'stderr_td_triplet.txt'

CalculationMode = td
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

SpinComponents = spin_polarized

TDPropagator = aetrs
TDTimeStep = 0.004/eV
TDMaxSteps = 2500  # ~ 10.0/TDTimeStep

TDDeltaStrength = 0.01/angstrom
TDPolarizationDirection = 1
TDDeltaStrengthMode = kick_spin

Besides the [SpinComponents](https://www.octopus-code.org/documentation//13/variables/states/spincomponents) variable, the main difference is the type of perturbation that is applied to the system. By setting <code><a href=https://www.octopus-code.org/documentation//13/variables/time-dependent/response/tddeltastrengthmode>TDDeltaStrengthMode</a> = kick_spin</code>, the kick will have opposite sign for up and down states. Whereas the ordinary kick (`kick_density`) yields the response to a homogeneous electric field, *i.e.* the electric dipole response, this kick yields the response to a homogeneous magnetic field, *i.e.* the magnetic dipole response. Note however that only the spin degree of freedom is coupling to the field; a different calculation would be required to obtain the orbital part of the response. Only singlet excited states contribute to the spectrum with `kick_density`, and only triplet excited states contribute with `kick_spin`. We will see below how to use symmetry to obtain both at once with `kick_spin_and_density`.


The time requiered to create all runs are in the order of 40 minutes.

In [ ]:
!octopus

### Spectrum

When the propagation completes, we run the [oct-propagation_spectrum](https://www.octopus-code.org/documentation/13/manual/external_utilities/oct-propagation_spectrum) utility to obtain the spectrum.


In [ ]:
!oct-propagation_spectrum

In [ ]:
!mv cross_section_vector cross_section_vector_triplet

Recalculate the singlet for comparisment

In [ ]:
%%writefile inp

stdout = 'stdout_gs_singlet.txt'
stderr = 'stderr_gs_singlet.txt'

CalculationMode = gs
FromScratch = yes
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

In [ ]:
!octopus

In [ ]:
%%writefile inp

stdout = 'stdout_td_singlet.txt'
stderr = 'stderr_td_singlet.txt'

CalculationMode = td
FromScratch = yes
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

TDPropagator = aetrs
TDTimeStep = 0.0023/eV
TDMaxSteps = 4350  # ~ 10.0/TDTimeStep

TDDeltaStrength = 0.01/angstrom
TDPolarizationDirection = 1

The time requiered to create all runs are in the order of 5 minutes.

In [ ]:
!octopus

In [ ]:
!oct-propagation_spectrum

In [ ]:
with open("cross_section_vector_triplet") as csv_triplet:
    print("".join(csv_triplet.readlines()))

In [ ]:
!mv cross_section_vector cross_section_vector_singlet

In [ ]:
with open("cross_section_vector_singlet") as csv_singlet:
    print("".join(csv_singlet.readlines()))

## Comparison of absorption spectrum of methane calculated with time-propagation for singlets and triplets.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Energy (eV)")
ax.set_ylabel("Strength function (1/eV)")


df_singlet = pd.read_csv(
    "cross_section_vector_singlet",
    delimiter="     ",
    usecols=[0, 4],
    names=["Energy (eV)", "StrengthFunction"],
    skiprows=26,
    engine="python",
)


df_triplet = pd.read_csv(
    "cross_section_vector_triplet",
    delimiter="     ",
    usecols=[0, 4],
    names=["Energy (eV)", "StrengthFunction"],
    skiprows=26,
    engine="python",
)


df_triplet["StrengthFunction"] = -1 * df_triplet["StrengthFunction"]


df_singlet.plot(
    x="Energy (eV)",
    y="StrengthFunction",
    ax=ax,
    label="singlet",
    xlim=[7, 15],
    color="purple",
)


df_triplet.plot(
    x="Energy (eV)",
    y="StrengthFunction",
    ax=ax,
    label="triplet",
    xlim=[7, 20],
    color="green",
);

You can see that there are now separate columns for cross-section and strength function for each spin. The physically meaningful strength function for the magnetic excitation is given by `StrengthFunction(1)` - `StrengthFunction(2)` (since the kick was opposite for the two spins). (If we had obtained `cross_section_tensor`, then the trace in the second column would be the appropriate cross-section to consider.) We can plot and compare to the singlet results obtained before. You can see how this looks on the right. The first triplet transition is found at 9.05 eV, slightly lower energy than the lowest singlet transition.

If you are interested, you can also repeat the calculation for <code><a href=https://www.octopus-code.org/documentation//13/variables/time-dependent/response/tddeltastrengthmode>TDDeltaStengthMode</a> =TDDeltaStrengthMode </code> (the default) and confirm that the result is the same as for the non-spin-polarized calculation.

### Using symmetries of non-magnetic systems

As said before, methane is a non-magnetic system, that is, the up and down densities are the same and the magnetization density is zero everywhere:

$$
 \rho^{\uparrow}(\mathbf r) = \rho^{\downarrow}(\mathbf r)
$$

Note that it is not enough that the total magnetic moment of the system is zero as the previous condition does not hold for anti-ferromagnetic systems. The symmetry in the spin-densities can actually be exploited in order to obtain both the singlet and triplet spectra with a single calculation. This is done by using a special perturbation that is only applied to the spin up states.[$^1$](#first_reference)
To use this perturbation, we need to set <code><a href=https://www.octopus-code.org/documentation//13/variables/time-dependent/response/tddeltastrengthmode>TDDeltaStrengthMode</a> = kick_spin_and_density</code>. If you repeat the time-propagation with this kick, you should obtain a different `cross_section_vector` file containing both the singlet and triplet spectra. The singlets are given by `StrengthFunction(1)` + `StrengthFunction(2)`, while the triplets are given by `StrengthFunction(1)``StrengthFunction(2)`.


## Casida equation

The calculation of triplets with the `casida` mode for spin-polarized systems is currently not implement in **Octopus**. Nevertheless, just like for the time-propagation, we can take advantage that for non-magnetic systems the two spin are equivalent. In this case it allow us to calculate triplets without the need for a spin-polarized run. The effective kernels in these cases are:

$f_{\rm Hxc}^{\rm singlet} \left[ \rho \right] = f^{\uparrow}_{\rm Hxc} \left[ \rho ^{\uparrow} \right] + f^{\uparrow}_{\rm Hxc} \left[ \rho^{\downarrow} \right] = f_{\rm H} \left[ \rho \right] + f^{\uparrow}_{\rm xc} \left[ \rho ^{\uparrow} \right] + f^{\uparrow}_{\rm xc} \left[ \rho^{\downarrow} \right]$

$f_{\rm Hxc}^{\rm triplet} \left[ \rho \right] = f^{\uparrow}_{\rm Hxc} \left[ \rho ^{\uparrow} \right] - f^{\uparrow}_{\rm Hxc} \left[ \rho^{\downarrow} \right] = f^{\uparrow}_{\rm xc} \left[ \rho ^{\uparrow} \right] - f^{\uparrow}_{\rm xc} \left[ \rho^{\downarrow} \right]$

Therefore, we start by doing a ground-state and unoccupied states runs exactly as was done in the [Optical spectra from Casida tutorial](3_Optical_spectra_from_casida.ipynb). Then, do a Casida run with the following input file:


In [ ]:
%%writefile inp

stdout = 'stdout_gs_casida.txt'
stderr = 'stderr_gs_casida.txt'

CalculationMode = gs
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

In [ ]:
!octopus

## Unoccupied states

In [ ]:
%%writefile inp

stdout = 'stdout_unocc_casida.txt'
stderr = 'stderr_unocc_casida.txt'

CalculationMode = unocc
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

ExtraStates = 10

In [ ]:
!octopus

## Triplet: Casida

In [ ]:
%%writefile inp

stdout = 'stdout_triplet_casida.txt'
stderr = 'stderr_triplet_casida.txt'

CalculationMode = casida
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

ExtraStates = 10

CasidaCalcTriplet = yes

ExperimentalFeatures = yes

In [ ]:
!octopus

In [ ]:
!oct-casida_spectrum

In [ ]:
subprocess.run(
    ["mv spectrum.casida spectrum.casida_triplet && mv spectrum.casida_triplet .."],
    shell=True,
    cwd="casida",
)

## Singlet: Casida

In [ ]:
%%writefile inp

stdout = 'stdout_singlet_casida.txt'
stderr = 'stderr_singlet_casida.txt'

CalculationMode = casida
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

ExtraStates = 10

ExperimentalFeatures = yes

In [ ]:
!octopus

In [ ]:
!oct-casida_spectrum

In [ ]:
subprocess.run(
    ["mv spectrum.casida spectrum.casida_singlet && mv spectrum.casida_singlet .."],
    shell=True,
    cwd="casida",
)

### Comparison of absorption spectrum of methane calculated with the Casida equation for singlets and triplets.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Energy (eV)")
ax.set_ylabel("Strength function (1/eV)")


df_casida_singlet = pd.read_csv(
    "spectrum.casida_singlet",
    delimiter="  ",
    usecols=[0, 4],
    names=["Energy (eV)", "StrengthFunction"],
    skiprows=1,
    engine="python",
)


df_casida_triplet = pd.read_csv(
    "spectrum.casida_triplet",
    delimiter="  ",
    usecols=[0, 4],
    names=["Energy (eV)", "StrengthFunction"],
    skiprows=1,
    engine="python",
)


df_triplet["StrengthFunction"] = -1 * df_triplet["StrengthFunction"]


df_casida_singlet.plot(
    x="Energy (eV)",
    y="StrengthFunction",
    ax=ax,
    label="singlet",
    xlim=[7, 15],
    color="purple",
)


df_casida_triplet.plot(
    x="Energy (eV)",
    y="StrengthFunction",
    ax=ax,
    label="triplet",
    xlim=[0, 20],
    color="green",
);

# Comparison

### Comparison of triplet absorption spectrum of methane calculated with time-propagation and with the Casida equation.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Energy (eV)")
ax.set_ylabel("Strength function (1/eV)")


df_triplet["StrengthFunction"] = -1 * df_triplet["StrengthFunction"]


df_triplet.plot(
    x="Energy (eV)",
    y="StrengthFunction",
    ax=ax,
    label="time-propagation",
    xlim=[0, 20],
    color="purple",
)


df_casida_triplet.plot(
    x="Energy (eV)",
    y="StrengthFunction",
    ax=ax,
    label="Casida",
    xlim=[0, 20],
    color="green",
);

[Go to *6_Use_of_symmetries_in_optical_spectra_from_time_propagation.ipynb*](6_Use_of_symmetries_in_optical_spectra_from_time_propagation.ipynb)

## References

1. M.J.T. Oliveira, A. Castro, M.A.L. Marques, and A. Rubio, On the use of Neumann's principle for the calculation of the polarizability tensor of nanostructures, [J. Nanoscience and Nanotechnology]() 8 1-7 (2008);
<span id="first_reference"></span>